### Dataset A: edades categorizadas (chequear toda la notebook)

In [56]:
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, classification_report, f1_score
import gc


In [3]:
dataset_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'datasets')

##### Datos a usar
Dataset de llamados completo, limpio con reducción de variables hecha a mano: variables agrupadas por temática, variables eliminadas por poco informativas, variables reducidas: hecho lugar, llamado provincia, llamante vinculo, victima nacionalidad, y agresor conocido/no conocido. 


##### Pipeline

1. cargar dataset
2. reemplazar NS/NC en convive por NA
3. hacer modificaciones para svm. Considerar: Apply scikit's OneHotEncoder with the handle_unknown parameter set to "ignore"
4. separar NA de convive, serán el dataset "no visto"
5. separar features de target
6. separar train-test 
7. armar modelo
8. evaluar


##### Modificaiones para SVM

1. pasar datetime a timestamp y escalar --> OK
2. escalar la variable edad --> OK
3. pasar edad a categórica (en una columna distinta) --> OK
4. Encodeo:
   1. edad, momento del día y estación del año con un encoder ordinal
   2. escalar las features encodeadas con ordinal
   3. variables de SI/NO encodear para 1 y 0
   4. el resto con one hot encoder
5. borrar las categóricas para hacer más pequeño el dataset




In [20]:
# 1. cargar dataset


llamados= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v3.xlsx'), parse_dates=['llamado_fecha_hora'])



In [21]:
llamados.columns

Index(['llamado_fecha_hora', 'llamante_edad', 'llamante_genero',
       'caso_judicializado', 'victima_a_resguardo', 'victima_edad',
       'victima_genero', 'victima_discapacidad', 'victima_convive_agresor',
       'vs_tocamiento_sexual', 'vs_intento_tocamiento', 'vs_grooming',
       'vs_exhibicionismo', 'vs_obligacion_sacarse_fotos_pornograficas',
       'vs_acoso_sexual', 'vs_iniciacion_sexual_forzada_inducida',
       'vs_otra_forma_violencia_sexual', 'vs_no_sabe_no_contesta',
       'ofv_sentimiento_amenaza', 'ofv_amenaza_explicita',
       'ofv_violencia_fisica', 'ofv_enganio_seduccion', 'ofv_grooming',
       'ofv_otra_forma_violencia', 'ofv_no_sabe_no_contesta', 'fin_de_semana',
       'momento_dia', 'estacion_del_año', 'vs_explotacion_sexual_group',
       'vs_violacion_group', 'vs_tentativa_group',
       'agresor_conocido_no_conocido_red', 'vinculo_llamante_red',
       'hecho_lugar_red', 'llamado_provincia_red', 'victima_nacionalidad_red'],
      dtype='object')

In [22]:
# 2 reemplazar todos los NS/NC en convive por na

llamados.loc[:, 'victima_convive_agresor'] = llamados['victima_convive_agresor'].replace({'NS/NC': pd.NA})



3. Modificaciones para SVM



In [23]:
# 1. timestamp-fecha_hora encodeado y escalado
scaler = MinMaxScaler()
encoder = OrdinalEncoder()


llamados['timestamp_encoded'] = encoder.fit_transform(llamados[['llamado_fecha_hora']])
llamados['timestamp_encoded_scaled'] = scaler.fit_transform(llamados[['timestamp_encoded']])
    
llamados.timestamp_encoded_scaled

# sobra llamado_fecha_hora y timestamp_encoded

0        0.000000
1        0.000060
2        0.000121
3        0.000181
4        0.000181
           ...   
19138    0.999758
19139    0.999940
19140    0.999819
19141    0.999879
19142    0.999879
Name: timestamp_encoded_scaled, Length: 19143, dtype: float64

In [25]:
# 3. pasar edad a categórica

def categoria_edad (x):
    if (x >= 0) and (x <= 11) :
        return 'Niñez'
    elif (x >= 12) and (x <=18):
        return 'Adolescencia'
    elif (x >= 19) and (x <=30):
        return 'Juventud'
    elif (x>=31) and (x<=65) :
        return 'Adultez'
    elif x>=66:
        return 'Vejez'
    else:
        return 'NS/NC'



llamados['victima_edad_cat'] = \
llamados.victima_edad.apply(categoria_edad)
llamados['llamante_edad_cat'] = \
llamados.llamante_edad.apply(categoria_edad)


# sobra: victima_edad y llamante_edad  llamado_fecha_hora y timestamp_encoded  


In [26]:

llamados['victima_edad_cat_encoded'] = encoder.fit_transform(llamados[['victima_edad_cat']])

In [27]:
llamados['llamante_edad_cat_encoded'] = encoder.fit_transform(llamados[['llamante_edad_cat']])

In [28]:
# 4. Variables categóricas que serán encodeadas con ordinal: edad cat (ordinal), momento día (ordinal), 
# estación del año (ordinal).
# Además de encodearlas con ordinal, luego voy a escalar ese encoding ordinal. 
# FUENTE Encoding_Methods_for_Categorical_Data.pdf

# 4.1 pasar edad_cat, momento_dia (V5) y estacion_del_año (V5) a dummy con un encoder ordinal

llamados['momento_encoded'] = encoder.fit_transform(llamados[['momento_dia']])
llamados['estacion_encoded'] = encoder.fit_transform(llamados[['estacion_del_año']])



#sobra victima_edad_cat y llamante_edad_cat victima_edad y llamante_edad  llamado_fecha_hora y timestamp_encoded momento_dia y estacion_del_año 

In [29]:

# 4.2 escalar las variables ordinales edad, momento del día y estación


llamados['victima_edad_encoded_scaled'] = scaler.fit_transform(llamados[['victima_edad_cat_encoded']])
llamados['llamante_edad_encoded_scaled'] = scaler.fit_transform(llamados[['llamante_edad_cat_encoded']])
llamados['momento_encoded_scaled'] = scaler.fit_transform(llamados[['momento_encoded']])
llamados['estacion_encoded_scaled'] = scaler.fit_transform(llamados[['estacion_encoded']])


#sobra victima_edad_cat_encoded y llamante_edad_cat_encoded, momento_encoded y estacion_encoded
# momento_dia y estacion_del_año victima_edad_cat y llamante_edad_cat victima_edad y llamante_edad  llamado_fecha_hora y timestamp_encoded  


In [34]:
# droppear las columnas que ya sé que no voy a usar
llamados.drop(['victima_edad_cat_encoded', 'llamante_edad_cat_encoded', 
                 'momento_encoded', 'estacion_encoded', 'momento_dia',
                 'estacion_del_año', 'victima_edad_cat', 
                 'llamante_edad_cat', 'victima_edad', 'llamante_edad',  
                 'llamado_fecha_hora', 'timestamp_encoded'], axis=1, inplace=True) 

In [35]:
objetos = {}
for column in llamados.columns:
    if llamados[column].dtypes == object:
        objetos[column] = llamados[column].unique()




In [36]:
objetos

{'llamante_genero': array(['Femenino', 'Masculino', 'NS/NC', 'Transgénero'], dtype=object),
 'caso_judicializado': array(['NS/NC', 'NO', 'SI'], dtype=object),
 'victima_a_resguardo': array(['SI', 'NO'], dtype=object),
 'victima_genero': array(['Femenino', 'Masculino', 'NS/NC', 'Transgénero'], dtype=object),
 'victima_discapacidad': array(['SI', 'NO', 'NS/NC'], dtype=object),
 'victima_convive_agresor': array(['SI', 'NO', <NA>], dtype=object),
 'vs_tocamiento_sexual': array(['SI', 'NO'], dtype=object),
 'vs_intento_tocamiento': array(['NO', 'SI'], dtype=object),
 'vs_grooming': array(['NO', 'SI'], dtype=object),
 'vs_exhibicionismo': array(['NO', 'SI'], dtype=object),
 'vs_obligacion_sacarse_fotos_pornograficas': array(['NO', 'SI'], dtype=object),
 'vs_acoso_sexual': array(['NO', 'SI'], dtype=object),
 'vs_iniciacion_sexual_forzada_inducida': array(['NO', 'SI'], dtype=object),
 'vs_otra_forma_violencia_sexual': array(['NO', 'SI'], dtype=object),
 'vs_no_sabe_no_contesta': array(['NO', '

In [42]:
# 4.3 Variables de SI/NO encodear con dummy
# Find columns that only contain 'SI' and 'NO'

si_no_columns = []
cat_mas_de_dos = []
for column in llamados.columns:
    if llamados[column].dtypes == object:
        if set(llamados[column].unique()).issubset({'SI', 'NO'}):
            si_no_columns.append(column)
        else:
            cat_mas_de_dos.append(column)
        



In [43]:
si_no_columns

['victima_a_resguardo',
 'vs_tocamiento_sexual',
 'vs_intento_tocamiento',
 'vs_grooming',
 'vs_exhibicionismo',
 'vs_obligacion_sacarse_fotos_pornograficas',
 'vs_acoso_sexual',
 'vs_iniciacion_sexual_forzada_inducida',
 'vs_otra_forma_violencia_sexual',
 'vs_no_sabe_no_contesta',
 'ofv_sentimiento_amenaza',
 'ofv_amenaza_explicita',
 'ofv_violencia_fisica',
 'ofv_enganio_seduccion',
 'ofv_grooming',
 'ofv_otra_forma_violencia',
 'ofv_no_sabe_no_contesta',
 'vs_explotacion_sexual_group',
 'vs_violacion_group',
 'vs_tentativa_group']

In [41]:
encodear_one_hot = ['llamante_genero',
 'caso_judicializado',
 'victima_genero',
 'victima_discapacidad',
 'agresor_conocido_no_conocido_red',
 'vinculo_llamante_red',
 'hecho_lugar_red',
 'llamado_provincia_red',
 'victima_nacionalidad_red']


In [44]:
# encodear si/no 

for col in si_no_columns:
    llamados[col] = llamados[col].map({'SI': 1, 'NO':0})

In [45]:
#encodear target
llamados['victima_convive_agresor'] = llamados['victima_convive_agresor'].map({'SI': 1, 'NO':0})

In [47]:
# encodear con one hot 

encoder = OneHotEncoder(sparse_output=True) # considerar drop first is sparse false

# Fit and transform the data
encoded_sparse = encoder.fit_transform(llamados[encodear_one_hot])

# Convert the sparse matrix to a dense array
encoded_array = encoded_sparse.toarray()

# Create a DataFrame with the encoded columns
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(encodear_one_hot))

# Drop the original columns that were one-hot encoded
llamados.drop(columns=encodear_one_hot, inplace=True)

# Concatenate the original DataFrame with the encoded DataFrame
llamados = pd.concat([llamados, encoded_df], axis=1)




In [48]:
# sacar todos los casos vacíos de convive y ponerlos en un nuevo dataset aparte, 
# será el test final a predecir con el mejor modelo

# Create a DataFrame with rows where 'convive' is NA
test_final = llamados[llamados['victima_convive_agresor'].isna()]

# Remove rows where 'convive' is NA from the original DataFrame
llamados = llamados.dropna(subset=['victima_convive_agresor'])


In [49]:
gc.collect()

108

In [52]:
llamados.isnull().values.any()

False

In [51]:
llamados.drop(['victima_edad_escalada', 'llamante_edad_escalada'], axis=1, inplace=True) 

In [57]:
# Separate features and target variable
X = llamados.drop(columns=['victima_convive_agresor'])
y = llamados['victima_convive_agresor']

# Stratified split
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Define the parameter grid for GridSearchCV
param_grid = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.1, 1, 10, 100],
    'gamma': ['scale', 'auto']
}

# Initialize the SVM model
svm = SVC()

# Perform GridSearchCV to find the best parameters
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation f1: ", grid_search.best_score_)
print("Test set f1: ", f1_score(y_test, y_pred, average='weighted'))
print("Classification report:\n", classification_report(y_test, y_pred))


Best parameters found:  {'C': 1, 'gamma': 'scale', 'kernel': 'poly'}
Best cross-validation f1:  0.8702039958656039
Test set f1:  0.8708323889876585
Classification report:
               precision    recall  f1-score   support

         0.0       0.90      0.95      0.93      2466
         1.0       0.72      0.54      0.62       551

    accuracy                           0.88      3017
   macro avg       0.81      0.75      0.77      3017
weighted avg       0.87      0.88      0.87      3017



In [ ]:
# Apply the best model to the subset of the original dataset
X_na = test_final.drop(columns=['victima_convive_agresor', 'llamante_edad_escalada', 'victima_edad_escalada'])
na_predictions = best_model.predict(X_na)

# Add the predictions to the na_convive_df DataFrame
test_final['victima_convive_agresor_pred'] = na_predictions

print("\nPredictions for NA 'convive' values:")
print(test_final)

In [214]:
test_final.victima_convive_agresor_pred.value_counts()

victima_convive_agresor_pred
0.0    3748
1.0     310
Name: count, dtype: int64

In [216]:
test_final.shape

(4058, 78)

In [218]:
llamados.shape

(15085, 75)

In [217]:
llamados.victima_convive_agresor.value_counts()

victima_convive_agresor
0.0    12329
1.0     2756
Name: count, dtype: int64

In [222]:
print('Proporción de SI en dataset original', llamados.victima_convive_agresor.value_counts()[1]/llamados.shape[0], 'Proporción de SI en test final',test_final.victima_convive_agresor_pred.value_counts()[1]/test_final.shape[0] )
print('Proporción de NO en dataset original', test_final.victima_convive_agresor_pred.value_counts()[1]/test_final.shape[0], 'Proporción de NO en test final', test_final.victima_convive_agresor_pred.value_counts()[0]/test_final.shape[0])


Proporción de SI en dataset original 0.18269804441498178 Proporción de SI en test final 0.0763923114834894
Proporción de NO en dataset original 0.8173019555850183 Proporción de NO en test final 0.9236076885165106
